In [2]:
import screencapture as sc

import tensorflow as tf
import numpy as np
from PIL import ImageGrab
import cv2, os, time, select


def network(x):
    linear = tf.layers.flatten(x)
    with tf.name_scope("Eyes"):
        enc1 = tf.layers.dense(linear, 16*16*4)
        enc2 = tf.layers.dense(enc1, 8*8*4)
        enc3 = tf.layers.dense(enc2, 4*4*4)
    with tf.name_scope("Brain"):
        dec1 = tf.layers.dense(enc3, 64*64*4)
    loss = tf.losses.mean_squared_error(labels=linear, predictions=dec1)
    train = tf.train.AdamOptimizer(0.0001).minimize(loss)
    return enc3, dec1, loss, train


scap=sc.ScreenCapturer(960,540,64,64)
round = 1000

with tf.Session() as sess:
    for i in range(4):
        screen=scap.get_gray()
        scap.save_pic(screen)

    scr_tf = [np.transpose(scap.screen_data, (1,2,0))]
    print('Start building network')
    x = tf.placeholder(tf.float64, shape=(None, 64, 64, 4))
    encoder, decoder, loss, train = network(x)
    img_tensor=tf.reshape(decoder,[-1,4,4,3])
    print('Network built.')
    sess.run(tf.initializers.global_variables())
    loss_ = sess.run(loss, {x: scr_tf})
    print('Initial loss: ',end='')
    print(loss_)
    print('Start training... total round = {0}'.format(round))

    for epoch in range(round):
        sess.run(train, {x:scr_tf})
        loss_=sess.run(loss, {x: scr_tf})
        if (epoch+1)%50 == 0:
            print('Epoch {0} completed, loss='.format(epoch+1),end='')
            print(loss_)
            
    print('Train complete')
    loss_ = sess.run(loss, {x: scr_tf})
    print('Loss:')
    print(loss_)

Start building network
Network built.
Initial loss: 18039.795
Start training... total round = 1000
Epoch 50 completed, loss=807.1893
Epoch 100 completed, loss=3.9737298
Epoch 150 completed, loss=0.019019011
Epoch 200 completed, loss=0.000109648376
Epoch 250 completed, loss=5.68843e-07
Epoch 300 completed, loss=3.013148e-09
Epoch 350 completed, loss=6.683401
Epoch 400 completed, loss=0.11095326
Epoch 450 completed, loss=0.0011871638
Epoch 500 completed, loss=13.238415
Epoch 550 completed, loss=2.7160504
Epoch 600 completed, loss=22.7778
Epoch 650 completed, loss=16.738445
Epoch 700 completed, loss=2.199925
Epoch 750 completed, loss=0.022223799
Epoch 800 completed, loss=12.734135
Epoch 850 completed, loss=6.2880774
Epoch 900 completed, loss=0.28050035
Epoch 950 completed, loss=5.134034
Epoch 1000 completed, loss=0.35589898
Train complete
Loss:
0.35589898
